1) IMPORT LIBRARIES

In [ ]:
!python -m spacy download el_core_news_sm 

In [ ]:
! pip install dateparser         #Parse dates written using natural language into python

import spacy

import requests                            
from bs4 import BeautifulSoup as soup  

import pandas as pd
import matplotlib.pyplot as plt  
%matplotlib inline
plt.style.use('ggplot')
import numpy as np 
import glob 
import re    
from datetime import datetime
import dateparser
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


2) READ THE CSV FILE & DATA PRE-PROCESSING

In [ ]:
from google.colab import drive      
drive.mount('/content/gdrive')

        #reading the csv 
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Scrape_efsyn.csv") #parse_dates=['Time'], date_parser= dateparser.parse]  

Mounted at /content/gdrive


In [ ]:
df.head(10)

,Title,Summary,Time
0,"Τους ψήφισαν, τους επέτρεψαν, είναι καιρός να ...",Εδώ και λίγες μέρες με τις ψήφους μόνο της Ν....,"08.07.2021, 16:00"
1,"Τώρα και για τους εκπαιδευτικούς, μόρια αντί γ...",Τη λειτουργία «εκπαιδευτικών ομίλων» μετά τη ...,"05.07.2021, 10:27"
2,Οι κρίσεις και το δόγμα του ατομισμού,Σκεφτείτε τι θα γινόταν αν ο καύσωνας της προ...,"05.07.2021, 04:00"
3,Το ασφαλιστικό των millennials,Είναι αναγκαίο να υπάρξει αλλαγή υποδείγματος...,"04.07.2021, 10:26"
4,Της Αλλοπάρ*,"Οι συλλεκτικές, διασκεδαστικές και, γενικά, α...","03.07.2021, 06:00"
5,Τσίπρας: Νέα αρχή με στόχο τη νίκη απέναντι στ...,Λάβρος κατά της κυβέρνησης και του πρωθυπουργ...,"02.07.2021, 21:31"
6,Γεννηματά: «Για την πανδημία δεν αρκεί η επικο...,Τον επικοινωνιακό τρόπο με τον οποίο επιλέγει...,"02.07.2021, 06:00"
7,"Ιδιος ο στόχος, αλλάζει μόνο ο τρόπος",Στόχος της Δεξιάς είναι η δημιουργία ενός καθ...,"28.06.2021, 16:00"
8,Υποτιμώντας μέχρι θανάτου τη γυναίκα,Η ρίζα του κακού βρίσκεται στον τρόπο εγγραφή...,"27.06.2021, 15:41"
9,Στην Ελλάδα το να ονομαστείς σκηνοθέτης θεωρεί...,"Ερμηνεύει τον Οθέλλο στο έργο του Σέξπιρ, επι...","27.06.2021, 11:12"


In [ ]:
#converting 'Time' from str to datetime
df['Time'] = pd.to_datetime(df.Time , format='%d.%m.%Y, %H:%M')
df['Time']

In [ ]:
df['year'] = pd.DatetimeIndex(df['Time']).year
df['month'] = pd.DatetimeIndex(df['Time']).month
df['day'] = pd.DatetimeIndex(df['Time']).day
df

,Title,Summary,Time,year,month,day
0,"Τους ψήφισαν, τους επέτρεψαν, είναι καιρός να ...",Εδώ και λίγες μέρες με τις ψήφους μόνο της Ν....,2021-07-08 16:00:00,2021,7,8
1,"Τώρα και για τους εκπαιδευτικούς, μόρια αντί γ...",Τη λειτουργία «εκπαιδευτικών ομίλων» μετά τη ...,2021-07-05 10:27:00,2021,7,5
2,Οι κρίσεις και το δόγμα του ατομισμού,Σκεφτείτε τι θα γινόταν αν ο καύσωνας της προ...,2021-07-05 04:00:00,2021,7,5
3,Το ασφαλιστικό των millennials,Είναι αναγκαίο να υπάρξει αλλαγή υποδείγματος...,2021-07-04 10:26:00,2021,7,4
4,Της Αλλοπάρ*,"Οι συλλεκτικές, διασκεδαστικές και, γενικά, α...",2021-07-03 06:00:00,2021,7,3
...,...,...,...,...,...,...
295,Ηλιόπουλος: Επικίνδυνη η Ν.Δ. – κεντρική η μάχ...,Ο εκπρόσωπος Τύπου του ΣΥΡΙΖΑ-ΠΣ εστίασε στο ...,2021-04-17 17:25:00,2021,4,17
296,Βαρουφάκης: Το εργασιακό νομοσχέδιο του Κ. Χατ...,Κριτική στο νομοσχέδιο του Κ. Χατζηδάκη που θ...,2021-04-17 17:00:00,2021,4,17
297,ΣΥΡΙΖΑ: Η κυβέρνηση Μητσοτάκη οδηγεί σε ανεξέλ...,«Η κυβέρνηση έχει κηρύξει πόλεμο στους εργαζο...,2021-04-16 12:39:00,2021,4,16
298,Πώς έγινε... άνθρακες ο «χρυσός» του Πρίνου;,Σε δεινή θέση βρίσκεται η εταιρεία Energean K...,2021-04-16 12:13:00,2021,4,16


In [ ]:
#Set datetime as index
df.set_index('Time', inplace= True)
df.head(5)

In [ ]:
df.dtypes

In [ ]:
#converting data type from object to string
df['Summary']= df['Summary'].astype(str)

In [ ]:
#converting data type from object to string
df['Title']= df['Title'].astype(str)

In [ ]:
#print the whole heading (need some cleaning)
pd.set_option("display.max_colwidth", -1)
print(df['Title'])

In [ ]:
#Cleaning the text
def cleanTxt(text):
  text = re.sub(r'[^\w\s]', '', text)   #Removing puncuations
  text = text.lower()                     #Converting letters to lowercase
  text = text.replace('\xa0','')       #Removing \xa0 character(it means space)  
  text = text.replace('\t','')        #Removing \t character(similar to the indentation function in the document,Tab key) 
  text = text.replace('\n','')                                                                                          
  #text = re.sub('\d+', '', text)      #Removing numbers             
# Return the cleaned text
  return text
#Clean the tweets
df['Summary'] = df['Summary'].apply(cleanTxt)
df['Title'] =  df['Title'].apply(cleanTxt)
#Show the cleaned tweets
df.head(100)

,Title,Summary,year,month,day
Time,,,,,
2021-07-08 16:00:00,τους ψήφισαν τους επέτρεψαν είναι καιρός να αντιδράσουν,εδώ και λίγες μέρες με τις ψήφους μόνο της νδ αποτελεί νόμο του κράτους το νέο εργασιακό νομοσχέδιο ή καλύτερα η ταφόπλακα των δικαιωμάτων και των κεκτημένων των εργαζομένων,2021,7,8
2021-07-05 10:27:00,τώρα και για τους εκπαιδευτικούς μόρια αντί για αμοιβή,τη λειτουργία εκπαιδευτικών ομίλων μετά τη λήξη του ημερήσιου ωρολογίου προγράμματος διδασκαλίας προβλέπει το νέο νομοσχέδιο του υπουργείου παιδείας,2021,7,5
2021-07-05 04:00:00,οι κρίσεις και το δόγμα του ατομισμού,σκεφτείτε τι θα γινόταν αν ο καύσωνας της προηγούμενης εβδομάδας διαρκούσε περισσότερο και οι θερμοκρασίες ήταν υψηλότερες οποιαδήποτε δραστηριότητα σε εξωτερικό χώρο που θα συνεπάγεται σωματική,2021,7,5
2021-07-04 10:26:00,το ασφαλιστικό των millennials,είναι αναγκαίο να υπάρξει αλλαγή υποδείγματος στο ασφαλιστικό αλλά και σε πολλούς ακόμη τομείς της δημόσιας ζωής η εισαγωγή της ανταποδοτικότητας στις επικουρικές συντάξεις η δημιουργία δηλαδή ενός ατομικού,2021,7,4
2021-07-03 06:00:00,της αλλοπάρ,οι συλλεκτικές διασκεδαστικές και γενικά απολαυστικές εκπομπές στο ραδιόφωνο του σκαϊ,2021,7,3
...,...,...,...,...,...
2021-06-11 13:10:00,κυρανάκης θα μπορείτε τώρα να δουλεύετε υπερωρίες για να πάτε στον γιατρό,αδιανόητα επιχειρήματα για ακόμη μια φορά από τον βουλευτή της νδ ο οποίος είτε αγνοεί πλήρως τα ισχύοντα εργασιακά δικαιώματα ή αδιαφορεί παντελώς για αυτά,2021,6,11
2021-06-11 09:41:00,συλλαλητήρια και στην κρήτη μαζικό οχι στο νέο εργασιακό νομοσχέδιο,με μεγάλη συμμετοχή και με βασικό αίτημα την απόσυρση του νέου εργασιακού νομοσχεδίου του υπουργείου εργασίας πραγματοποιήθηκαν την πέμπτη τα συλλαλητήρια στην κρήτη στο πλαίσιο της 24ωρης απεργίας της 10ης,2021,6,11
2021-06-11 07:50:00,στοπειθαρχικό στέλνει ηεσηεατους απεργοσπάστες τουσκαϊ,tον γενικό διευθυντή ενημέρωσης του σκαϊ βασίλη παπαδρόσο και τον δημοσιογράφο νίκο υποφάντη παραπέμπει η εσηεα και ζητά τη διερεύνηση τυχόν άλλων υπευθύνων,2021,6,11


In [ ]:
#convert from int to string(month name)
df['month'] = pd.to_datetime(df['month'], format='%m').dt.month_name().str.slice(stop=3)

4) NATURAL LANGUAGE PROCESSING

In [ ]:
#Greek language
nlp = spacy.load('el_core_news_sm')

In [ ]:
#Greek stopwords are included in this list
list(nlp.Defaults.stop_words)[0:10]

In [ ]:
GreekStopwords = nlp.Defaults.stop_words

In [ ]:
#Save the greek stopwords in a variable
NewStopWords = list(GreekStopwords)

In [ ]:
#Concatenate
title = df['Title'].str.cat(sep = ' ')
print(len(title), 'items')
title

In [ ]:
summary = df['Summary'].str.cat(sep = ' ')
print(len(summary) , 'items')
summary

In [ ]:
#merge summary + title together
whole_text = summary + title
print(len(whole_text) , 'items')
whole_text

5) LEMMATIZATION: THE PROCESS OF RETURNING THE BASE OR DICTIONARY FORM OF A WORD (LEMMA)

In [ ]:
#print only the lemma for every word
full_text = nlp(whole_text)
for token in full_text:       
    print(token.lemma_)

In [ ]:
#Modify the full text by replacing every word with its lemma.
final_text = ' '.join(token.lemma_ for token in full_text)
final_text[5:200]

In [ ]:
#Store the clear/lemmatized full text in a variable
full_doc = nlp(final_text)
full_doc

In [ ]:
clear_text = ' '.join(token.lemma_ for token in full_doc)
clear_text[:500]
